In [3]:
import requests
from bs4 import BeautifulSoup
import functions as f
import pandas as pd
import time
from time import sleep
import random
import io 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
import json
import os.path

In [5]:
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
headers = {'User-Agent': user_agent}

In [10]:
movies1 = "data//movies1.html"
movies2 = "data//movies2.html"
movies3 = "data//movies3.html"

#CREATE DF
df_movies1 = f.createDfFromHtml(movies1)
df_movies2 = f.createDfFromHtml(movies2)
df_movies3 = f.createDfFromHtml(movies3)

In [ ]:
#DOWNLOAD HTML FILES
f.saveMoviesHtml(df_movies1)
f.saveMoviesHtml(df_movies2)
f.saveMoviesHtml(df_movies3)

In [13]:
df_movies = pd.concat([df_movies1, df_movies2, df_movies3])
df_movies.reset_index(inplace=True)
df_movies.drop(columns=["index"], inplace=True)

In [8]:
##### create .tsv files
f.createTsv()

In [10]:
f.createVocabulary(stop_words)

In [38]:
vocabulary = f.load_vocabulary()

In [65]:
#### create inverted index
inverted_index = f.createInvertedIndex(vocabulary)

In [60]:
#Load inverted index 1
inverted_index = f.load_inverted_index_1()

In [95]:
def search_1(query, top = 10):
    return f.top_k(f.search_1(query, vocabulary, stop_words, inverted_index), df_movies, top, no_score=True)

In [96]:
search_1('dog house')

,Title,Intro,URL
0,The Killing of a Chinese Bookie,"The Killing of a Chinese Bookie is a 1976 American neo-noir[2] crime film written and directed by John Cassavetes and starring Ben Gazzara.[3][4][5][6][7] A rough and gritty film, this is the second o ...",https://en.wikipedia.org/wiki/The_Killing_of_a_Chinese_Bookie
1,Wild Oranges,"Wild Oranges is a 1924 American silent film directed by King Vidor.[1] On January 12, 2010 the film had its first home video release, on the Warner Archive DVD series. ...",https://en.wikipedia.org/wiki/Wild_Oranges
2,Alice in Wonderland (1903 film),Alice in Wonderland is a 1903 British silent film directed by Cecil Hepworth and Percy Stow.[1] Only one copy of the original film is known to exist. The British Film Institute (BFI) partially restor ...,https://en.wikipedia.org/wiki/Alice_in_Wonderland_(1903_film)
3,The Strange Affair of Uncle Harry,"The Strange Affair of Uncle Harry is a 1945 American film noir drama directed by Robert Siodmak and starring George Sanders as an aging bachelor who looks after his two sisters, one of whom tries to s ...",https://en.wikipedia.org/wiki/The_Strange_Affair_of_Uncle_Harry
4,The Thin Man Goes Home,The Thin Man Goes Home is a 1945 film directed by Richard Thorpe. It is the fifth of the six Thin Man films starring William Powell and Myrna Loy as Dashiell Hammett's dapper ex-private detective Nick ...,https://en.wikipedia.org/wiki/The_Thin_Man_Goes_Home
5,Johnny One-Eye,"Johnny One-Eye is a 1950 film noir crime film directed by Robert Florey starring Pat O'Brien, Wayne Morris, Dolores Moran and Gayle Reed. ...",https://en.wikipedia.org/wiki/Johnny_One-Eye
6,Go West (1925 film),"Go West (1925) is a silent movie starring Buster Keaton.[1][2][3] Keaton portrays Friendless, who travels west to try to make his fortune. Once there, he tries his hand at bronco-busting, cattle wran ...",https://en.wikipedia.org/wiki/Go_West_(1925_film)
7,The Hound of the Baskervilles (1939 film),The Hound of the Baskervilles is a 1939 mystery film based on the novel The Hound of the Baskervilles by Sir Arthur Conan Doyle. It was directed by Sidney Lanfield and released by 20th Century-Fox.[1] ...,https://en.wikipedia.org/wiki/The_Hound_of_the_Baskervilles_(1939_film)
8,Princess O'Rourke,"Princess O'Rourke is a 1943 American romantic comedy film[2][3]directed and written by Norman Krasna (in Krasna's directorial debut), and starring Olivia de Havilland, Robert Cummings and Charles Cob ...",https://en.wikipedia.org/wiki/Princess_O%27Rourke
9,The Family Secret (1924 film),"The Family Secret is a 1924 American silent drama film directed by William A. Seiter and featuring child star Baby Peggy. It is based on Editha's Burglar, a story by Frances Hodgson Burnett first publ ...",https://en.wikipedia.org/wiki/The_Family_Secret_(1924_film)


# Inverted index 2

In [101]:
### TO REMOVE
from imp import reload 
reload(f)

<module 'functions' from '/Users/adm-vp/anaconda3/JUPYTER_NOTES/HW3_G24/functions.py'>

In [46]:
inverted_index_2 = f.load_inverted_index_2()
doc_norm = f.load_doc_norm()

In [87]:
def search_2(query, top = 10):
    return f.top_k(f.search_with_tfidf(query, vocabulary, inverted_index_2, doc_norm), df_movies, top)

In [88]:
search_2('world war II', 20)

,Score,Title,Intro,URL
0,0.56,Ghost of the China Sea,Ghost of the China Sea is a 1958 American war film released by Columbia Pictures co-written by Charles B. Griffith set during World War II. It was the last movie directed by Fred F. Sears who died of ...,https://en.wikipedia.org/wiki/Ghost_of_the_China_Sea
1,0.37,Battle Zone (film),"Battle Zone is a 1952 Korean War war film.[1] Sequences of the film were shot at Camp Pendleton, California.[2] ...",https://en.wikipedia.org/wiki/Battle_Zone_(film)
2,0.33,When Hell Broke Loose,When Hell Broke Loose is a 1958 World War II war film directed by Kenneth G. Crane and starring Charles Bronson. It was co-written by Ib Melchior. ...,https://en.wikipedia.org/wiki/When_Hell_Broke_Loose
3,0.33,The Sailor Takes a Wife,The Sailor Takes a Wife is a 1945 American romantic comedy film directed by Richard Whorf and starring Robert Walker and June Allyson. ...,https://en.wikipedia.org/wiki/The_Sailor_Takes_a_Wife
4,0.32,"Captain Carey, U.S.A.","Captain Carey, U.S.A. is a 1950 crime film directed by Mitchell Leisen and starring Alan Ladd, Wanda Hendrix. An American returns to post–World War II Italy to bring a traitor to justice. The film was ...","https://en.wikipedia.org/wiki/Captain_Carey,_U.S.A."
5,0.31,Prelude to War,"Prelude to War is the first film of Frank Capra's Why We Fight film series, commissioned by the Office of War Information (OWI) and George C. Marshall. It was made to educate American troops of the ne ...",https://en.wikipedia.org/wiki/Prelude_to_War
6,0.29,Destroyer (1943 film),Destroyer is a 1943 American war film directed by William A. Seiter and starring Edward G. Robinson. ...,https://en.wikipedia.org/wiki/Destroyer_(1943_film)
7,0.28,The Mountain Road,"The Mountain Road is a 1960 war film starring James Stewart and directed by Daniel Mann. Set in China and based on the 1958 novel of the same name by journalist-historian Theodore H. White, the film ...",https://en.wikipedia.org/wiki/The_Mountain_Road
8,0.27,The Yanks Are Coming (1942 film),The Yanks Are Coming is a 1942 American patriotic musical film from Poverty Row studio Producers Releasing Corporation directed by Alexis Thurn-Taxis.[1] ...,https://en.wikipedia.org/wiki/The_Yanks_Are_Coming_(1942_film)
9,0.27,Purple Heart Diary,"Purple Heart Diary is a 1951 drama directed by Richard Quine, produced by Sam Katzman and released by Columbia Pictures. It stars Frances Langford and Judd Holdren.[1] ...",https://en.wikipedia.org/wiki/Purple_Heart_Diary


## Task 3

In [100]:
#  Create new index and norm for movie names
name_index, name_norm = f.create_inverted_index_2([3], vocabulary, norm_file='name_norm.json', index_file='name_index.json')

TypeError: create_inverted_index_2() got multiple values for argument 'norm_file'

In [ ]:
def search_3(query, top = 10):
    #  as_heap = False ==> returns scores as dictionary {document_id : score}
    #  strict_terms_filter = False ==> tells function to use not_strict_filter_documents
    name_results = search_with_tfidf(query, name_index, name_norm, as_heap=False, strict_terms_filter=False)
    
    #  additional_score will be added to scores with the same document id
    search_results = search_with_tfidf(query, inverted_index_2, doc_norm, additional_score=name_results)
    return top_k(search_results, top)
    
search_3('disney dog', 10)